In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from rtseg.utils.param_io import load_params
from pathlib import Path
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread
from rtseg.forkplot import compute_forkplot_stats
import matplotlib.patches as patches
%matplotlib qt5

In [3]:
from rtseg.segmentation import LiveNet, get_live_model, live_segment

/home/pk/Documents/rtclient/.venv/lib/python3.11/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: '2.0.2' (you have '2.0.0'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
params_path = Path("/home/pk/Documents/rtseg/rtseg/resources/reference_params/reference_linux.yaml")
params = load_params(params_path, ref_type='expt')

In [5]:
phase_path = Path('/home/pk/Documents/rtclient/rtclient/resources/test_images/phase_dummy.tiff')
fluor_path = Path('/home/pk/Documents/rtclient/rtclient/resources/test_images/fluor_dummy.tiff')
phase_img = imread(phase_path).astype('float32')
fluor_img = imread(fluor_path)
datapoint = {
    'phase': phase_img,
    'fluor': fluor_img,
    'position': 1,
    'timepoint': 0,
}

In [6]:
live_net = get_live_model(params)

In [7]:
seg_result = live_segment(datapoint, live_net, params)

In [8]:
plt.figure()
plt.imshow(seg_result['seg_mask'])
plt.show()

In [9]:
plt.figure()
plt.imshow(phase_img)
plt.show()

In [10]:
fig, ax = plt.subplots()
ax.imshow(phase_img, cmap='gray')
for row in seg_result['barcode_locations']:
    rect = patches.Rectangle((row[0], row[1]), row[2] - row[0], row[3] - row[1], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
plt.show()

In [11]:
seg_result

{'phase': array([[ 9191,  9630,  9750, ...,  9215,  9126,  8947],
        [ 9543,  9264,  9978, ...,  8928,  8727,  9221],
        [ 9442,  9854,  9719, ...,  9542,  9192,  9502],
        ...,
        [ 9132, 10096, 10089, ..., 10937, 10458, 10586],
        [ 9916,  9450,  9832, ..., 10307, 10652, 11273],
        [ 9788,  9445, 10127, ..., 10408, 10779, 10595]], dtype=uint16),
 'fluor': array([[251, 201, 266, ..., 241, 200, 229],
        [222, 220, 231, ..., 211, 221, 218],
        [215, 220, 243, ..., 218, 222, 226],
        ...,
        [253, 260, 225, ..., 207, 244, 213],
        [275, 195, 236, ..., 201, 239, 236],
        [273, 254, 220, ..., 195, 215, 218]], dtype=uint16),
 'position': 1,
 'timepoint': 0,
 'barcode_locations': [array([5.03924990e+00, 2.19385025e+02, 9.49055023e+01, 1.00268695e+03,
         6.10479295e-01, 0.00000000e+00], dtype=float32),
  array([1.4597417e+03, 3.3374533e+02, 1.5652759e+03, 1.1148585e+03,
         6.3969195e-01, 0.0000000e+00], dtype=float32),
  

In [12]:
from rtseg.dotdetect import compute_dot_coordinates

In [13]:
params.Hardware.device

'cuda:0'

In [14]:
dots= compute_dot_coordinates(seg_result['fluor'], seg_result['seg_mask'], params)

In [15]:
dots

{'raw_coords': array([[ 783.14738426, 1487.63868987],
        [1265.01042238, 3027.50923386],
        [ 277.35200941, 2939.82150615],
        ...,
        [1014.54571118,  100.4640608 ],
        [ 168.5046111 ,   19.1091004 ],
        [ 169.431315  ,    9.47310662]]),
 'rotated_coords': array([[1547.36131013,  783.14738426],
        [   7.49076614, 1265.01042238],
        [  95.17849385,  277.35200941],
        ...,
        [2934.5359392 , 1014.54571118],
        [3015.8908996 ,  168.5046111 ],
        [3025.52689338,  169.431315  ]])}

In [16]:
def plot_dots(fluor_image, dots):
    plt.figure()
    plt.imshow(fluor_image)
    plt.plot(dots[:, 1], dots[:, 0], 'ro')
    plt.show()

In [17]:
plot_dots(seg_result['seg_mask'], dots['raw_coords'])

### Computing Internal coordinates to plot

In [18]:
from rtseg.cells.utils import regionprops_custom

In [19]:
cell_mask_rot = np.rot90(seg_result['seg_mask']).copy()

In [20]:
props = regionprops_custom(cell_mask_rot)

In [21]:
trap_locations = seg_result['trap_locations_list']

In [22]:
trap_locations

[116,
 218,
 320,
 422,
 524,
 628,
 728,
 830,
 933,
 1035,
 1137,
 1239,
 1340,
 1444,
 1583,
 1686,
 1787,
 1891,
 1996,
 2095,
 2196,
 2299,
 2404,
 2506,
 2606,
 2708,
 2811,
 2913]

In [23]:
plt.figure()
plt.imshow(np.rot90(seg_result['seg_mask']))
plt.show()

In [24]:
fork_stats = compute_forkplot_stats(seg_result['seg_mask'], dots['rotated_coords'], trap_locations=trap_locations)

In [25]:
import pandas as pd

In [26]:
fork_stats_table = pd.DataFrame(fork_stats)

In [27]:
fork_stats_table

,position,timepoint,trap,cell_label,area,length,normalization_counts,internal_coord,normalized_internal_x,bbox,global_coords,local_coords
0,0,0,4,15,368.0,30.025607,2,"(11.98922317624821, 1.6399408567633893)",0.399300,"(2490, 227, 2511, 258)","(2498.0897671568628, 237.50857843137254)","(8.089767156862763, 10.508578431372541)"
1,0,0,4,15,368.0,30.025607,2,"(9.788336731681527, -3.865848233606922)",0.326000,"(2490, 227, 2511, 258)","(2504.0020677771395, 237.7408385985066)","(14.002067777139473, 10.740838598506599)"
2,0,0,8,16,596.0,55.741157,4,"(48.05295137060893, 4.746203548365416)",0.862073,"(2082, 228, 2104, 286)","(2083.5879702709813, 277.0554788213628)","(1.587970270981259, 49.05547882136278)"
3,0,0,8,16,596.0,55.741157,4,"(37.3826746952392, 4.079160304717635)",0.670648,"(2082, 228, 2104, 286)","(2084.875520523498, 265.62076145151696)","(2.8755205234979258, 37.620761451516955)"
4,0,0,8,16,596.0,55.741157,4,"(25.4714595987455, 1.3097386295719315)",0.456960,"(2082, 228, 2104, 286)","(2090.5891111187348, 254.25496586503826)","(8.589111118734763, 26.25496586503826)"
...,...,...,...,...,...,...,...,...,...,...,...,...
1020,0,0,23,355,582.0,47.668006,1,"(25.005121986290117, 0.34552260922799044)",0.524568,"(535, 1198, 552, 1247)","(541.6917005993503, 1222.8776593311068)","(6.69170059935027, 24.87765933110677)"
1021,0,0,19,356,371.0,37.364992,2,"(18.373922048488232, 0.4941400365225665)",0.491742,"(944, 1204, 963, 1241)","(952.8927276323598, 1221.6494362761257)","(8.892727632359765, 17.649436276125698)"
1022,0,0,19,356,371.0,37.364992,2,"(28.80098371269993, 2.6872475579280533)",0.770801,"(944, 1204, 963, 1241)","(954.2500640751815, 1232.1520717642034)","(10.250064075181513, 28.152071764203356)"
1023,0,0,25,357,455.0,44.946226,2,"(37.892924170826184, 3.064576871844503)",0.843072,"(331, 1215, 350, 1259)","(340.747684015496, 1252.2806131042614)","(9.74768401549602, 37.28061310426142)"


In [32]:
len(fork_stats_table)

1025

In [39]:
fork_stats_table.to_csv('forks.csv', mode='a', index=False, header=False)

In [40]:
df = pd.read_csv('forks.csv')

In [41]:
df

,position,timepoint,trap,cell_label,area,length,normalization_counts,internal_coord,normalized_internal_x,bbox,global_coords,local_coords
0,0,0,4,15,368.0,30.025607,2,"(11.98922317624821, 1.6399408567633893)",0.399300,"(2490, 227, 2511, 258)","(2498.0897671568628, 237.50857843137254)","(8.089767156862763, 10.508578431372541)"
1,0,0,4,15,368.0,30.025607,2,"(9.788336731681527, -3.865848233606922)",0.326000,"(2490, 227, 2511, 258)","(2504.0020677771395, 237.7408385985066)","(14.002067777139473, 10.740838598506599)"
2,0,0,8,16,596.0,55.741157,4,"(48.05295137060893, 4.746203548365416)",0.862073,"(2082, 228, 2104, 286)","(2083.5879702709813, 277.0554788213628)","(1.587970270981259, 49.05547882136278)"
3,0,0,8,16,596.0,55.741157,4,"(37.3826746952392, 4.079160304717635)",0.670648,"(2082, 228, 2104, 286)","(2084.875520523498, 265.62076145151696)","(2.8755205234979258, 37.620761451516955)"
4,0,0,8,16,596.0,55.741157,4,"(25.4714595987455, 1.3097386295719315)",0.456960,"(2082, 228, 2104, 286)","(2090.5891111187348, 254.25496586503826)","(8.589111118734763, 26.25496586503826)"
...,...,...,...,...,...,...,...,...,...,...,...,...
3070,0,0,23,355,582.0,47.668006,1,"(25.005121986290117, 0.34552260922799044)",0.524568,"(535, 1198, 552, 1247)","(541.6917005993503, 1222.8776593311068)","(6.69170059935027, 24.87765933110677)"
3071,0,0,19,356,371.0,37.364992,2,"(18.373922048488232, 0.4941400365225665)",0.491742,"(944, 1204, 963, 1241)","(952.8927276323598, 1221.6494362761257)","(8.892727632359765, 17.649436276125698)"
3072,0,0,19,356,371.0,37.364992,2,"(28.80098371269993, 2.6872475579280533)",0.770801,"(944, 1204, 963, 1241)","(954.2500640751815, 1232.1520717642034)","(10.250064075181513, 28.152071764203356)"
3073,0,0,25,357,455.0,44.946226,2,"(37.892924170826184, 3.064576871844503)",0.843072,"(331, 1215, 350, 1259)","(340.747684015496, 1252.2806131042614)","(9.74768401549602, 37.28061310426142)"


In [41]:
len(df)

3077

In [58]:
df.iloc[1025]

position                              position
timepoint                            timepoint
trap                                      trap
cell_label                          cell_label
area                                      area
length                                  length
normalization_counts      normalization_counts
internal_coord                  internal_coord
normalized_internal_x    normalized_internal_x
bbox                                      bbox
global_coords                    global_coords
local_coords                      local_coords
Name: 1025, dtype: object

In [54]:
trap_locations_left = [loc-40 for loc in trap_locations]
trap_locations_right = [loc+40 for loc in trap_locations]

In [58]:
print(trap_locations_left)
print(trap_locations_right)

[76, 178, 280, 382, 484, 588, 688, 790, 893, 995, 1097, 1199, 1300, 1404, 1543, 1646, 1747, 1851, 1956, 2055, 2156, 2259, 2364, 2466, 2566, 2668, 2771, 2873]
[156, 258, 360, 462, 564, 668, 768, 870, 973, 1075, 1177, 1279, 1380, 1484, 1623, 1726, 1827, 1931, 2036, 2135, 2236, 2339, 2444, 2546, 2646, 2748, 2851, 2953]


In [51]:
props[0].centroid

(2933.2462686567164, 167.36567164179104)

In [52]:
import bisect

In [61]:
for prop in props:
    left_index = bisect.bisect_left(trap_locations_left, prop.centroid[0])
    right_index = bisect.bisect_right(trap_locations_right, prop.centroid[0])
    print(prop.centroid[0], trap_locations_left[left_index-1], trap_locations_right[right_index], left_index-1, right_index)
    if left_index - 1 == right_index:
        prop.trap = right_index
    else:
        prop.trap = None
    prop.left_index = left_index
    prop.right_index = right_index

2933.2462686567164 2873 2953 27 27
2063.690476190476 2055 2135 19 19
2722.548672566372 2668 2748 25 25
1464.5013698630137 1404 1484 13 13
2207.8502673796793 2156 2236 20 20
1414.361963190184 1404 1484 13 13
2705.904593639576 2668 2748 25 25
1902.078313253012 1851 1931 17 17
1991.3779193205944 1956 2036 18 18
1600.5 1543 1623 14 14
2398.1688311688313 2364 2444 22 22
1347.2350119904077 1300 1380 12 12
2599.437984496124 2566 2646 24 24
800.2452830188679 790 870 7 7
2498.6603260869565 2466 2546 23 23
2091.6593959731545 2055 2135 19 19
1235.9347826086957 1199 1279 11 11
2804.16765578635 2771 2851 26 26
1149.392857142857 1097 1177 10 10
2906.8090815273476 2873 2953 27 27
1036.440281030445 995 1075 9 9
2190.878306878307 2156 2236 20 20
430.7031630170316 382 462 3 3
1680.3676470588234 1646 1726 15 15
1589.2236842105262 1543 1623 14 14
523.6628477905074 484 564 4 4
2295.34179357022 2259 2339 21 21
626.2301458670988 588 668 5 5
2599.326530612245 2566 2646 24 24
1887.576 1851 1931 17 17
1782.9785

In [56]:
props[0].left_index

28

In [57]:
props[0].right_index

27